In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')    
    
type(sc)

pyspark.context.SparkContext

In [13]:
data = sqlContext.read.format('csv').options(header='true', inferSchema='true').load('.../events.csv')
display(data)


DataFrame[timestamp: timestamp, event: string, person: string, url: string, sku: string, model: string, condition: string, storage: string, color: string, skus: string, search_term: string, staticpage: string, campaign_source: string, search_engine: string, channel: string, new_vs_returning: string, city: string, region: string, country: string, device_type: string, screen_resolution: string, operating_system_version: string, browser_version: string]

In [14]:
type(data)

pyspark.sql.dataframe.DataFrame

In [15]:
data.columns

['timestamp',
 'event',
 'person',
 'url',
 'sku',
 'model',
 'condition',
 'storage',
 'color',
 'skus',
 'search_term',
 'staticpage',
 'campaign_source',
 'search_engine',
 'channel',
 'new_vs_returning',
 'city',
 'region',
 'country',
 'device_type',
 'screen_resolution',
 'operating_system_version',
 'browser_version']

In [16]:
data.take(5)

[Row(timestamp=datetime.datetime(2018, 5, 31, 23, 38, 5), event='ad campaign hit', person='0004b0a2', url='/comprar/iphone/iphone-5s', sku=None, model=None, condition=None, storage=None, color=None, skus=None, search_term=None, staticpage=None, campaign_source='criteo', search_engine=None, channel=None, new_vs_returning=None, city=None, region=None, country=None, device_type=None, screen_resolution=None, operating_system_version=None, browser_version=None),
 Row(timestamp=datetime.datetime(2018, 5, 31, 23, 38, 5), event='visited site', person='0004b0a2', url=None, sku=None, model=None, condition=None, storage=None, color=None, skus=None, search_term=None, staticpage=None, campaign_source=None, search_engine=None, channel='Paid', new_vs_returning='New', city='Camaragibe', region='Pernambuco', country='Brazil', device_type='Smartphone', screen_resolution='360x640', operating_system_version='Android 6', browser_version='Chrome Mobile 39'),
 Row(timestamp=datetime.datetime(2018, 5, 31, 23,

In [17]:
dataRdd=data.rdd

# Ejercicio 1 : Encontrar cuál es el tipo de evento predominante.
### Mostramos 2 formas diferentes de resolver:


In [26]:
#--------------PRIMERA FORMA---------------
dataRdd.map(lambda x:(x[1],1))\
.reduceByKey(lambda x,y:x+y)\
.takeOrdered(11,lambda x:-x[1])

[('viewed product', 528931),
 ('brand listing', 98635),
 ('visited site', 87378),
 ('ad campaign hit', 82827),
 ('generic listing', 67534),
 ('searched products', 56073),
 ('search engine hit', 50957),
 ('checkout', 33735),
 ('staticpage', 3598),
 ('conversion', 1172),
 ('lead', 448)]

In [27]:
#--------------SEGUNDA FORMA-------------------
dataRdd.map(lambda x:(x[1],1))\
.reduceByKey(lambda x,y:x+y)\
.reduce(lambda x,y:x if x[1]>y[1] else y)

('viewed product', 528931)

# Ejercicio 2: Encontrar el Top 5 de dispositivos más visitados dentro de la categoría Smartphones.


In [32]:
dataRdd.map(lambda x: x[5]).distinct().collect()

['Motorola Moto X Style',
 'Quantum YOU',
 None,
 'Sony Xperia M4 Aqua Dual',
 'LG G4 Stylus HDTV H540T',
 'Asus Zenfone Go',
 'Motorola Moto G5S Plus',
 'Samsung Galaxy Core 2 Duos',
 'Sony Xperia Z5',
 'Samsung Galaxy J2 Prime TV',
 'Samsung Galaxy S4 i9500',
 'Sony Xperia Z5 Premium',
 'Motorola Moto G1 3G',
 'Motorola Moto G5 ',
 'Samsung Galaxy J5 PRO',
 'Samsung Galaxy Note 4',
 'Samsung Galaxy Note 3',
 'iPad 3 Wi-Fi + 4G',
 'LG Nexus 5 D821',
 'Quantum Muv Up',
 'Motorola Moto G3 HDTV',
 'iPad Mini Wi-Fi + 4G',
 'Asus Live',
 'Samsung Galaxy S5 Duos',
 'Motorola Moto G4 Plus',
 'iPhone 6 Plus',
 'LG K4',
 'Samsung Galaxy Gran 2 Duos TV',
 'LG G4 H818P',
 'Samsung Galaxy Gran Prime 3G Duos',
 'Motorola Moto G4 DTV',
 'Sony Xperia Z3 ',
 'Samsung Galaxy A7 2017',
 'Samsung Galaxy Tab E 7 Wi-Fi + 3G',
 'Sony Xperia Z3 Compact',
 'Samsung Galaxy Gran Neo Plus Duos',
 'Samsung Galaxy J7 2016 Metal',
 'Samsung Galaxy Tab A 2016 10.1  W-Fi + 4G',
 'Samsung Galaxy Core Plus Duos TV',
 

* **primero filtramos por evento que sera "viewed product: EL usuario visita una pagina de producto"**
* **sacamos la cantidad de visitas que tuvieron cada uno de los modelos**
* **hallamos los 5 mejores**

In [34]:
dataRdd.filter(lambda x: x[1]=="viewed product")\
.map(lambda x:(x[5],1)).reduceByKey(lambda x,y: x+y)\
.takeOrdered(5,lambda x: -x[1])

[('iPhone 6', 47538),
 ('iPhone 5s', 42045),
 ('iPhone 6S', 36930),
 ('iPhone 7', 26330),
 ('Samsung Galaxy S7 Edge', 23062)]